# Headers

In [28]:
# File system manangement
import time, psutil, os

# Mathematical functions
import math

# Data manipulation
import numpy as np
import pandas as pd

# Plotting and visualization
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as mpatches

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
from matplotlib import cm
from mpl_toolkits.mplot3d.axes3d import get_test_data


import numpy as np
from sklearn import preprocessing, neighbors, svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import RocCurveDisplay

# Loading files
Keep in mind I am only loading the training.zip here and then having a 80:20 split for train and test. One might as well also use the test.zip

In [29]:
df_train = pd.read_csv("training.zip", compression='zip')
df_train.head()
df_train["Label"] = df_train["Label"].map({"s": 1, "b": 0})

df_train_signal = df_train[df_train.Label == 1]
df_train_background = df_train[df_train.Label == 0]

In [30]:
X = np.array(df_train.drop(columns=['EventId', 'Label', 'Weight']).astype(float))
y = np.array(df_train['Label'])
X=X[:15000]
y=y[:15000]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Basic SVM example

In [31]:
# clf = svm.SVC(probability=True)
# clf.fit(X_train, y_train)
# confidence = clf.score(X_test, y_test)
# print(confidence)

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Create dataset and dataloader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


# Define the neural network
class SimpleDNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleDNN, self).__init__()
        self.hidden = nn.Linear(input_dim, 64)
        self.relu = nn.ReLU()
        self.output = nn.Linear(64, 1)  # Binary classification

    def forward(self, x):
        x = self.relu(self.hidden(x))
        x = torch.sigmoid(self.output(x))  # sigmoid for binary classification
        return x


# Initialize model
input_dim = X_train.shape[1]
model = SimpleDNN(input_dim)

# Loss and optimizer
criterion = nn.BCELoss()  # Binary cross-entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad() # reset old gradients
        outputs = model(batch_X) # forward pass
        loss = criterion(outputs, batch_y) # compute loss
        loss.backward() # backward pass (autograd computes gradients)
        optimizer.step() # update weights
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

print("Model architecture:\n", model, "\n")
# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        predicted = (outputs > 0.5).float()
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

    print(f"Accuracy on test set: {100 * correct / total:.2f}%")

Epoch 1/10, Loss: 13.6940
Epoch 2/10, Loss: 0.7329
Epoch 3/10, Loss: 0.7238
Epoch 4/10, Loss: 0.7217
Epoch 5/10, Loss: 0.7142
Epoch 6/10, Loss: 0.6247
Epoch 7/10, Loss: 0.6239
Epoch 8/10, Loss: 0.6859
Epoch 9/10, Loss: 0.6678
Epoch 10/10, Loss: 0.5888
Model architecture:
 SimpleDNN(
  (hidden): Linear(in_features=30, out_features=64, bias=True)
  (relu): ReLU()
  (output): Linear(in_features=64, out_features=1, bias=True)
) 

Accuracy on test set: 72.47%


In [35]:
class TwoLayerDNN(nn.Module):
    def __init__(self, input_dim):
        super(TwoLayerDNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)   # first hidden layer y=x.W^T+b
        self.fc2 = nn.Linear(64, 32)          # second hidden layer
        self.output = nn.Linear(32, 1)        # output layer
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = torch.sigmoid(self.output(x))     # sigmoid for binary classification
        return x

# Initialize model
input_dim = X_train.shape[1]
model = SimpleDNN(input_dim)

# Loss and optimizer
criterion = nn.BCELoss()  # Binary cross-entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad() # reset old gradients
        outputs = model(batch_X) # forward pass
        loss = criterion(outputs, batch_y) # compute loss
        loss.backward() # backward pass (autograd computes gradients)
        optimizer.step() # update weights
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

print("Model architecture:\n", model, "\n")
# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        predicted = (outputs > 0.5).float()
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

    print(f"Accuracy on test set: {100 * correct / total:.2f}%")


Epoch 1/10, Loss: 35.1514
Epoch 2/10, Loss: 15.3058
Epoch 3/10, Loss: 0.9318
Epoch 4/10, Loss: 0.8563
Epoch 5/10, Loss: 0.7347
Epoch 6/10, Loss: 0.7530
Epoch 7/10, Loss: 0.7270
Epoch 8/10, Loss: 0.6758
Epoch 9/10, Loss: 0.7151
Epoch 10/10, Loss: 0.7601
Model architecture:
 SimpleDNN(
  (hidden): Linear(in_features=30, out_features=64, bias=True)
  (relu): ReLU()
  (output): Linear(in_features=64, out_features=1, bias=True)
) 

Accuracy on test set: 72.77%


```text
Conv1: 7x7, 64, stride=2, padding=3
↓
BatchNorm
↓
ReLU
↓
MaxPool: 3x3, stride=2, padding=1
↓
+--------------------+
| Layer1:             |
| [Conv3x3,64 ->      |
|  Conv3x3,64] x2     |
| Residual blocks      |
+--------------------+
↓
+--------------------+
| Layer2:             |
| [Conv3x3,128 ->     |
|  Conv3x3,128] x2    |
| Residual blocks      |
| (downsample at first|
|  block with stride2)|
+--------------------+
↓
+--------------------+
| Layer3:             |
| [Conv3x3,256 ->     |
|  Conv3x3,256] x2    |
| Residual blocks      |
| (downsample at first|
|  block with stride2)|
+--------------------+
↓
+--------------------+
| Layer4:             |
| [Conv3x3,512 ->     |
|  Conv3x3,512] x2    |
| Residual blocks      |
| (downsample at first|
|  block with stride2)|
+--------------------+
↓
Average Pool (7x7)
↓
Fully Connected Layer (512 → num_classes, e.g., 1000 for ImageNet)
↓
Softmax (for classification)
```